In [1]:
import pandas as pd
import numpy as np
import random

In [3]:
df_app = pd.read_csv('application_train.csv')
df_bureau = pd.read_csv('bureau.csv')
df_bureau_balance = pd.read_csv('bureau_balance.csv')

# EAD

In [4]:
status_list = ['0','1','2','3','4','5']
df_bureau_balance = df_bureau_balance.query('STATUS in @status_list')
df_bureau_balance['STATUS'] = df_bureau_balance['STATUS'].astype('int')
df_bureau_balance = df_bureau_balance.set_index('SK_ID_BUREAU')

In [5]:
min_date = pd.DataFrame(data = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min())
min_date.columns = ['min_date']
min_default_date = pd.DataFrame(data = df_bureau_balance.query('STATUS in (4,5)').groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].min())
min_default_date.columns = ['min_default_date']
data_info = min_date.join(min_default_date, how='inner')
data_info = data_info.join(df_bureau.set_index('SK_ID_BUREAU')[['SK_ID_CURR','AMT_CREDIT_SUM']])
data_info['calc_annuity'] = data_info['AMT_CREDIT_SUM']/-data_info['min_date']
data_info['EAD'] = data_info['AMT_CREDIT_SUM'] - round(data_info['calc_annuity']) * (-data_info['min_date'] + data_info['min_default_date'])
data_info = data_info.reset_index()
df_ead = df_app.join(data_info.set_index('SK_ID_CURR'), how='inner', on='SK_ID_CURR').drop(['SK_ID_BUREAU','min_date','min_default_date','TARGET'], axis=1)
df_ead.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_SUM,calc_annuity,EAD
145,100167,Cash loans,F,N,Y,0,180000.0,427500.0,46170.0,427500.0,...,0,0.0,0.0,0.0,0.0,0.0,1.0,202500.00,6750.000000,141750.00
183,100213,Cash loans,F,N,N,2,49500.0,278460.0,19930.5,225000.0,...,0,0.0,0.0,0.0,1.0,0.0,0.0,36828.00,944.307692,22668.00
191,100221,Cash loans,F,N,Y,0,225000.0,684054.0,77494.5,630000.0,...,0,0.0,0.0,0.0,0.0,1.0,4.0,86012.55,5734.170000,11470.55
191,100221,Cash loans,F,N,Y,0,225000.0,684054.0,77494.5,630000.0,...,0,0.0,0.0,0.0,0.0,1.0,4.0,23184.00,282.731707,19505.00
483,100557,Cash loans,F,Y,Y,0,225000.0,1320975.0,43785.0,1183500.0,...,0,0.0,0.0,0.0,0.0,1.0,2.0,549000.00,6695.121951,488745.00


# LGD

In [6]:
def make_status(status_bki):
    if status_bki > 4:
        result = 4
    else:
        result = status_bki
    return result

In [7]:
df_bureau_balance_LGD = df_bureau_balance.copy()
df_bureau_balance_LGD['STATUS'] = df_bureau_balance_LGD.apply(lambda x: make_status(x.STATUS), axis=1)
df_bureau_balance_LGD.head()

,MONTHS_BALANCE,STATUS
SK_ID_BUREAU,,
5715448,-9,0
5715448,-10,0
5715448,-14,0
5715448,-15,0
5715448,-16,0


In [8]:
print(df_bureau_balance_LGD['MONTHS_BALANCE'].unique())
dates_list = list(df_bureau_balance_LGD['MONTHS_BALANCE'].unique())
dates_list.sort()

[ -9 -10 -14 -15 -16 -17 -18 -19  -6  -7  -8 -11 -12 -13 -20 -21 -22 -23
 -24 -25 -26 -27 -28 -29 -30 -34 -35 -36 -37 -38 -39 -40 -41 -42 -43 -44
 -45 -46 -47 -48 -49 -50 -51 -53   0  -1  -2  -3  -4  -5 -61 -31 -32 -33
 -59 -60 -62 -63 -64 -83 -84 -85 -86 -87 -88 -89 -90 -70 -71 -72 -73 -74
 -75 -76 -77 -65 -66 -67 -68 -69 -52 -54 -55 -56 -57 -58 -91 -92 -93 -94
 -95 -78 -80 -81 -79 -82 -96]


In [9]:
# Получаем помесячные матрицы перехода клиентов по бакетам просрочки

for date in dates_list:    
    if date < 0:
        
        print('Матрица перехода из месяца ' + str(date) + ' в месяц ' + str(date+1))
    
        # Получаем статистику по распределению клиентов по бакетам просрочки за текущий и следующий месяцы
        df_date = df_bureau_balance_LGD.query('MONTHS_BALANCE == @date')
        df_next_date = df_bureau_balance_LGD.query('MONTHS_BALANCE == @date + 1')    
        df_two_dates = df_date.join(df_next_date, how='inner', lsuffix='_this_date', rsuffix='_next_date')
        df_two_dates_stat = df_two_dates.groupby(['STATUS_this_date','STATUS_next_date'])['STATUS_next_date'].count()
        df_two_dates_stat = pd.DataFrame(df_two_dates_stat)
        df_two_dates_stat.columns = ['cnt']
        df_two_dates_stat.reset_index(inplace=True)         
        #display(df_two_dates_stat)
        
        # Получаем статистику по распределению клиентов по бакетам просрочки за текущий месяц
        df_this_date_stat = df_two_dates_stat.groupby(['STATUS_this_date'])['cnt'].sum()
        df_this_date_stat = pd.DataFrame(df_this_date_stat)
        df_this_date_stat.columns = ['cnt']
        #df_this_date_stat.reset_index(inplace=True)         
        #display(df_this_date_stat)
        
        # Получаем статистику по распределению переходов в бакеты просрочки на следующий месяц
        
        df_two_dates_stat['ratio'] = 0.0
        
        for i, row in df_two_dates_stat.iterrows():
            this_date = row['STATUS_this_date']
            this_cnt = row['cnt']
            all_cnt = df_this_date_stat.loc[this_date].values[0]
            df_two_dates_stat.at[i,'ratio'] = round(this_cnt / all_cnt,4)
        
        #display(df_two_dates_stat)
        
        # Представляем статистику по распределению в удобном виде
        
        df_two_dates_pivot = df_two_dates_stat.pivot(index = 'STATUS_this_date', columns = 'STATUS_next_date', values = 'ratio')
        df_two_dates_pivot.fillna(0, inplace=True)
        exec('mtrx_' + str(abs(date)) + ' = df_two_dates_pivot.copy()')
        display(df_two_dates_pivot)

Матрица перехода из месяца -96 в месяц -95


STATUS_next_date,0,1,2,3,4
STATUS_this_date,,,,,
0,0.9752,0.0223,0.0019,0.0001,0.0004
1,0.4432,0.4356,0.1174,0.0038,0.0000
2,0.2717,0.0870,0.2391,0.3913,0.0109
3,0.1081,0.0270,0.0270,0.1081,0.7297
4,0.0483,0.0207,0.0069,0.0000,0.9241


Матрица перехода из месяца -95 в месяц -94


STATUS_next_date,0,1,2,3,4
STATUS_this_date,,,,,
0,0.9737,0.0239,0.0018,0.0001,0.0005
1,0.4443,0.4268,0.1272,0.0017,0.0000
2,0.3158,0.0702,0.2193,0.3947,0.0000
3,0.1220,0.0488,0.0488,0.0244,0.7561
4,0.0303,0.0000,0.0061,0.0061,0.9576


Матрица перехода из месяца -94 в месяц -93


...

STATUS_next_date,0,1,2,3,4
STATUS_this_date,,,,,
0,0.9747,0.0250,0.0002,0.0000,0.0001
1,0.5954,0.3690,0.0345,0.0007,0.0004
2,0.3160,0.1991,0.1948,0.2814,0.0087
3,0.1882,0.0706,0.0706,0.0824,0.5882
4,0.0146,0.0021,0.0021,0.0021,0.9791


Матрица перехода из месяца -2 в месяц -1


STATUS_next_date,0,1,2,3,4
STATUS_this_date,,,,,
0,0.9726,0.0270,0.0002,0.0000,0.0002
1,0.5779,0.3881,0.0338,0.0002,0.0000
2,0.2778,0.1481,0.2176,0.3426,0.0139
3,0.0959,0.0274,0.0137,0.1096,0.7534
4,0.0197,0.0031,0.0000,0.0021,0.9751


Матрица перехода из месяца -1 в месяц 0


STATUS_next_date,0,1,2,3,4
STATUS_this_date,,,,,
0,0.9743,0.0253,0.0002,0.0000,0.0001
1,0.5857,0.3705,0.0419,0.0017,0.0002
2,0.3148,0.2083,0.1759,0.2870,0.0139
3,0.1392,0.0380,0.0886,0.1013,0.6329
4,0.0224,0.0031,0.0000,0.0010,0.9735


In [10]:
# Теперь усредняем полученные матрицы в одну, чтобы получить среднюю месячную матрицу миграции

# Сохраняем струткру других матриц, но обнуляем элементы
final_month_mtrx = pd.DataFrame(0.0, index=mtrx_1.index, columns=mtrx_1.columns)

# Бежим в цикле по всем элементам всех матриц

for i in mtrx_1.index:
    for c in mtrx_1.columns:
        sum_el = 0
        cnt_el = 0
        for date in dates_list:            
            if date < 0:               
                exec('mtrx_date = mtrx_' + str(abs(date)) + '.copy()')
                
                if mtrx_date.shape[0] == 5 and mtrx_date.shape[1] == 5: 
                    mtrx_el = mtrx_date.at[i,c]
                    sum_el = sum_el + mtrx_el
                    cnt_el = cnt_el + 1
                else:
                    sum_el = sum_el + 0
                    cnt_el = cnt_el + 0
      
        # Для каждого элемента финальной месячной матрицы считаем среднее по всем матрицам по соответствующим элементам
        if cnt_el == 0:
            final_month_mtrx.at[i,c] = 0.0
        else:
            final_month_mtrx.at[i,c] = round(sum_el/cnt_el,4)

final_month_mtrx

STATUS_next_date,0,1,2,3,4
STATUS_this_date,,,,,
0,0.9757,0.0232,0.0008,0.0000,0.0001
1,0.5193,0.3998,0.0794,0.0013,0.0003
2,0.2835,0.1637,0.2026,0.3419,0.0083
3,0.1357,0.0618,0.0554,0.1233,0.6238
4,0.0266,0.0064,0.0047,0.0036,0.9587


In [19]:
id_qty = 10000

df_generate_balance = pd.DataFrame(columns = ['id','dt_1'])

for i in range(0,id_qty):
    rnd = random.choices(list(final_month_mtrx.index), weights = final_month_mtrx[0])[0]
    df_generate_balance.loc[i] = [1,rnd]

df_generate_balance.groupby('dt_1')['id'].count()

dt_1
0    5188
1    2572
2    1416
3     713
4     111
Name: id, dtype: int64

In [20]:
for new_date in range(2,31):
    df_generate_balance['dt_' + str(new_date)] = 0.0   
    for row_id in range(0,id_qty):
        old_bucket = df_generate_balance.at[row_id,'dt_' + str(new_date-1)]
        rnd = random.choices(list(final_month_mtrx.index), weights = final_month_mtrx.loc[old_bucket])[0]
        df_generate_balance.at[row_id,'dt_' + str(new_date)] = rnd

df_generate_balance.groupby('dt_30')['id'].count()

dt_30
0.0    8901
1.0     387
2.0      56
3.0      22
4.0     634
Name: id, dtype: int64

In [24]:
df_generate_balance[['id','dt_1','dt_30']].groupby(['dt_1','dt_30'])['id'].count()

dt_1  dt_30
0     0.0      4852
      1.0       201
      2.0        28
      3.0         9
      4.0        98
1     0.0      2358
      1.0        91
      2.0        18
      3.0         5
      4.0       100
2     0.0      1133
      1.0        56
      2.0         7
      3.0         3
      4.0       217
3     0.0       487
      1.0        32
      2.0         3
      3.0         5
      4.0       186
4     0.0        71
      1.0         7
      4.0        33
Name: id, dtype: int64